<a href="https://colab.research.google.com/github/Chiriviki/congreso/blob/master/2.-%20Crea_corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.- Creación del corpus

<a target="_blank" href="https://colab.research.google.com/github/Chiriviki/congreso/blob/master/2.-%20Crea_corpus.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Este cuaderno toma los textos extraídos y trata de:
- Extraer metadatos y anexarlos a cada intervención.
- Limpiar los textos.
- Cruzar los nombres de políticos con el partido político al que pertenecen.
- Seleccionar las intervenciones y ls campos que forman parte del cospus final.


## Config

Este cuaderno funciona a partir de la versión 5 de Altair

In [1]:
!pip install altair==5.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.4/477.4 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: altair
    Found existing installation: altair 4.2.2
    Uninstalling altair-4.2.2:
      Successfully uninstalled altair-4.2.2


In [3]:
import os
import json
from datetime import datetime

import pandas as pd
import altair as alt

alt.data_transformers.disable_max_rows()

alt.__version__



'5.0.0'

## Carga corpus


In [4]:
!wget https://github.com/Chiriviki/congreso/raw/master/dataset.zip
!unzip dataset.zip -d "dataset"

--2023-05-28 09:56:59--  https://github.com/Chiriviki/congreso/raw/master/dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Chiriviki/congreso/master/dataset.zip [following]
--2023-05-28 09:56:59--  https://raw.githubusercontent.com/Chiriviki/congreso/master/dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57078988 (54M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  54.43M   198MB/s    in 0.3s    

2023-05-28 09:56:59 (198 MB/s) - ‘dataset.zip’ saved [57078988/57078988]

Archive:  dataset.zip
  inflating: dataset/Anexo_Diputados_

In [5]:
carpeta = 'dataset/corpus_v1'
 
corpus = []
for filename in os.listdir(carpeta):
    f = os.path.join(carpeta, filename)
    
    if os.path.isfile(f):
        with open(f, "r", encoding="latin1") as f_r:
            f_json = json.load(f_r)
            corpus.append(f_json)

            

In [6]:
corpus[0]

{'datos': 'DS. Congreso de los Diputados, Pleno y Dip. Perm., núm. 244, de 15/02/2023',
 'intervenciones': [{'name1': 'PRESIDENTA',
   'name2': None,
   'span': [9651, 9673],
   'texto': ' Muy buenos días, señorías.\nSe reanuda la sesión. Ocupen sus escaños, por favor. (Rumores).\nContinuamos con las preguntas dirigidas al Gobierno conforme al orden remitido por este. Preguntas dirigidas al señor presidente del Gobierno. Pregunta de la diputada doña Concepción Gamarra Ruiz-Clavijo, del Grupo Parlamentario Popular en\nel Congreso.'},
  {'name1': 'GAMARRA RUIZ-CLAVIJO',
   'name2': None,
   'span': [10017, 10049],
   'texto': ' Gracias, presidenta.\nSeñor Sánchez, eliminar el delito de sedición y abaratar la malversación le ha salido fatal. No nos ha homologado con ningún país europeo, ha desprotegido al Estado -como le ha sentenciado por unanimidad el Tribunal Supremo- y, además, no\nha conseguido contentar ni tan siquiera a sus socios independentistas.\nPor otro lado, ayer se cumplían 

## Extraer metadatos 

Los metadatos se encuentran a en una cadena de caracetres. La siguiente función extrae los distintos metadatos de esta cadena.


In [7]:


# Extraer los datos de la sesión

def extract_data(data_str):
    
    # Separa los elementos por coma
    data_splitted = data_str.split(",")
    
    # Los dos primeros campo los cogemos tal cual
    data ={ "cámara": data_splitted[0],
          "organismo": data_splitted[1]}
    
    # Extrae el número de sesión
    num = int(data_splitted[2][5:])    
    data["numero"] = num
    
    # Extrae fecha - se queda como str para dejarlo a gusto del consumidor
    fecha = data_splitted[3][4:]
    data["fecha"] = fecha
    return data

extract_data(corpus[0]["datos"])

{'cámara': 'DS. Congreso de los Diputados',
 'organismo': ' Pleno y Dip. Perm.',
 'numero': 244,
 'fecha': '15/02/2023'}

Con estos datos ya podemos crear una estrutura tabular.

In [8]:
dataset_df = pd.DataFrame([extract_data(sesion["datos"]) | {"orden_interv":i, "name1": interv["name1"], "name2": interv["name2"], "texto":interv["texto"]} for sesion in corpus for i, interv in enumerate(sesion["intervenciones"])])
        

In [9]:
dataset_df.head()

,cámara,organismo,numero,fecha,orden_interv,name1,name2,texto
0,DS. Congreso de los Diputados,Pleno y Dip. Perm.,244,15/02/2023,0,PRESIDENTA,None,"Muy buenos días, señorías.\nSe reanuda la ses..."
1,DS. Congreso de los Diputados,Pleno y Dip. Perm.,244,15/02/2023,1,GAMARRA RUIZ-CLAVIJO,None,"Gracias, presidenta.\nSeñor Sánchez, eliminar..."
2,DS. Congreso de los Diputados,Pleno y Dip. Perm.,244,15/02/2023,2,PRESIDENTA,None,Señor presidente del Gobierno.
3,DS. Congreso de los Diputados,Pleno y Dip. Perm.,244,15/02/2023,3,PRESIDENTE DEL GOBIERNO,Sánchez Pérez-Castejón,"Muchas gracias, señora presidenta.\nSeñoría, ..."
4,DS. Congreso de los Diputados,Pleno y Dip. Perm.,244,15/02/2023,4,GAMARRA RUIZ-CLAVIJO,None,"Sí, señor Sánchez, usted me contesta lo mismo..."


## Limpieza
Lo único que necesaitamos hacer es reemplazar los saltos de líneas por espacios.

In [10]:
dataset_df["texto"] = dataset_df["texto"].str.replace("\n", " ")
dataset_df.head()

,cámara,organismo,numero,fecha,orden_interv,name1,name2,texto
0,DS. Congreso de los Diputados,Pleno y Dip. Perm.,244,15/02/2023,0,PRESIDENTA,None,"Muy buenos días, señorías. Se reanuda la sesi..."
1,DS. Congreso de los Diputados,Pleno y Dip. Perm.,244,15/02/2023,1,GAMARRA RUIZ-CLAVIJO,None,"Gracias, presidenta. Señor Sánchez, eliminar ..."
2,DS. Congreso de los Diputados,Pleno y Dip. Perm.,244,15/02/2023,2,PRESIDENTA,None,Señor presidente del Gobierno.
3,DS. Congreso de los Diputados,Pleno y Dip. Perm.,244,15/02/2023,3,PRESIDENTE DEL GOBIERNO,Sánchez Pérez-Castejón,"Muchas gracias, señora presidenta. Señoría, u..."
4,DS. Congreso de los Diputados,Pleno y Dip. Perm.,244,15/02/2023,4,GAMARRA RUIZ-CLAVIJO,None,"Sí, señor Sánchez, usted me contesta lo mismo..."


In [11]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46062 entries, 0 to 46061
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cámara        46062 non-null  object
 1   organismo     46062 non-null  object
 2   numero        46062 non-null  int64 
 3   fecha         46062 non-null  object
 4   orden_interv  46062 non-null  int64 
 5   name1         46062 non-null  object
 6   name2         11123 non-null  object
 7   texto         46062 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.8+ MB


## Anexa partido

Los datos de los partidos se encuentran en dos archivos en formato de tabla extraídos de wikipedia.

A continuación se meustran las características que dispone. En nuestro caso solo necesitamos el nombre y el grupo político al que pertenece.

In [12]:
diputados = pd.read_csv("./dataset/Anexo_Diputados_de_la_XIV_legislatura_de_EspaB1a_2.csv")
diputados_baja = pd.read_csv("./dataset/Anexo_Diputados_de_la_XIV_legislatura_de_EspaB1a_causaronbaja.csv")

In [13]:
diputados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Nombre y apellidos  355 non-null    object 
 1   Grupo               0 non-null      float64
 2   Grupo.1             355 non-null    object 
 3   Lista               0 non-null      float64
 4   Lista.1             355 non-null    object 
 5   Circunscripción     355 non-null    object 
 6   Alta                355 non-null    object 
dtypes: float64(2), object(5)
memory usage: 19.5+ KB


In [14]:
diputados_baja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Nombre y apellidos  41 non-null     object 
 1   Grupo               0 non-null      float64
 2   Grupo.1             41 non-null     object 
 3   Lista               0 non-null      float64
 4   Lista.1             41 non-null     object 
 5   Circunscripción     41 non-null     object 
 6   Alta                41 non-null     object 
 7   Baja                41 non-null     object 
 8   Sustituto/a         41 non-null     object 
dtypes: float64(2), object(7)
memory usage: 3.0+ KB


Une ambos conjuntos de datos.

In [15]:
diputados_df = pd.concat([diputados, diputados_baja])[["Nombre y apellidos", "Grupo.1", "Lista.1", "Circunscripción", "Alta", "Baja"]]

El nombre aparece en formato "nombre, apellido1 apellido2". Para curzar los datos necesitamos únicamente los apellidos.

In [16]:
diputados_df[["Apellidos", "Nombre"]] = diputados_df["Nombre y apellidos"].str.split(", ", expand=True)
diputados_df.drop(columns="Nombre y apellidos", inplace=True)

In [17]:
diputados_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396 entries, 0 to 40
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Grupo.1          396 non-null    object
 1   Lista.1          396 non-null    object
 2   Circunscripción  396 non-null    object
 3   Alta             396 non-null    object
 4   Baja             41 non-null     object
 5   Apellidos        396 non-null    object
 6   Nombre           396 non-null    object
dtypes: object(7)
memory usage: 24.8+ KB


### Unión

Para unirlos se realiza las siguientes tareas:
- Se transofrma todo a mayúsculas.
- Se utiliza name2 si no es nulo, en tal caso se utiliza name1.
- Se curzan mediante Leftjoin

In [18]:
# Si name2 es nulo, lo sustituimos por name1

dataset_df['name2'] = dataset_df['name2'].fillna(dataset_df['name1'])

# Mayus
dataset_df["name2"] = dataset_df["name2"].str.upper()

diputados_df["Apellidos"] = diputados_df["Apellidos"].str.upper()

In [19]:
merged_partidos_name2 = dataset_df.merge(diputados_df, how="left", left_on="name2", right_on="Apellidos")

In [20]:
merged_partidos_name2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46859 entries, 0 to 46858
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cámara           46859 non-null  object
 1   organismo        46859 non-null  object
 2   numero           46859 non-null  int64 
 3   fecha            46859 non-null  object
 4   orden_interv     46859 non-null  int64 
 5   name1            46859 non-null  object
 6   name2            46859 non-null  object
 7   texto            46859 non-null  object
 8   Grupo.1          26519 non-null  object
 9   Lista.1          26519 non-null  object
 10  Circunscripción  26519 non-null  object
 11  Alta             26519 non-null  object
 12  Baja             1400 non-null   object
 13  Apellidos        26519 non-null  object
 14  Nombre           26519 non-null  object
dtypes: int64(2), object(13)
memory usage: 5.7+ MB


De las 48000 intervenciones, solo ha podido cruzar unas 26000. Pese a los erroes, el número de itnervenciones es suficiente. La mayoría de los cruces erróneos son de la presidenta del congreso.

In [21]:
errores = merged_partidos_name2[merged_partidos_name2["Grupo.1"].isna()]

In [22]:
errores["name2"].value_counts()

PRESIDENTA                                                     18687
CALVIÑO SANTAMARÍA                                               308
BOLAÑOS GARCÍA                                                   140
DARIAS SAN SEBASTIÁN                                             105
SÁNCHEZ JIMÉNEZ                                                   95
                                                               ...  
IPIÑAZAR MIRANDA                                                   1
BLANCO DE ANGULO                                                   1
SÁNCHEZ MARTÍN                                                     1
MINISTRO DE ASUNTOS EXTERIORES, UNIÓN EUROPEA Y COOPERACIÓN        1
BERMÚDEZ DE CASTRO                                                 1
Name: name2, Length: 229, dtype: int64

## Selección

Se va a tratar de seleccionar las intervenciones que van a ser objeto de estudio posteriormente. También se seleciconar un subconjunto de los campos.

### Selección de campos

Dejamos las columnas que nos interesan:
- Número de intervención
- Fecha de la sesion
- Nombre del parlamentario
- Grupo del parlamentario.
- Número de palabras de la intervención.

In [23]:
dataset_df = merged_partidos_name2[["numero", "fecha", "name2", "Grupo.1", "texto"]].rename(columns={"name2":"apellidos", "Grupo.1":"grupo"})
dataset_df["fecha"] = pd.to_datetime(dataset_df["fecha"], format="%d/%m/%Y")

### Selección de intervenciones

Se van a seleccionar únicamente las intervenciones de los siguientes partidos:
- PSOE
- Popular
- Vox
- Unidas Podemos
- ERC
- PNV

Se utiliza el nombre del grupo parlamentario, en lugar del partido. Por lo que vamos a sustituirlo por nombres o siglas conocidas.

In [24]:
dataset_df = dataset_df.replace(
    {"Confederal de Unidas Podemos-\nEn Comú Podem-Galicia en Común":"UP",
     "Vasco (EAJ-PNV)": "PNV",
    "Popular":"PP",
    "Socialista":"PSOE", 
     "Republicano":"ERC"})


dataset_df = dataset_df[dataset_df["grupo"].isin(["UP", "PNV", "PP", "PSOE", "ERC", "Vox"])]
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19773 entries, 1 to 46857
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   numero     19773 non-null  int64         
 1   fecha      19773 non-null  datetime64[ns]
 2   apellidos  19773 non-null  object        
 3   grupo      19773 non-null  object        
 4   texto      19773 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 926.9+ KB


### Excluir mesa del congreso

Si comprobamos el número de intervenciones por parlamentario, podemos ver que hay algunos con una gran cantidad de intervencioens (pej. Gomez de Celis). Esto es debido a que es vicepresidente del congreso y se ha cargado como diputado de su partido. 

Se prescinde de las intervenciones de todos los compnentes de la mesa del congreso. estos son:

- Batet Lamaña, Meritxell 
- Rodríguez Gómez de Celis, Alfonso
- Pastor Julián, Ana María
- Elizo Serrano, María Gloria 
- Gil Lázaro, Ignacio
- Pisarello Prados, Gerardo 
- Hernanz Costa, Sofía 
- Sánchez Serna, Javier 
- Navarro Lacoba, Carmen

In [25]:
componentes_mesa=[ "BATET LAMAÑA",
                  "RODRÍGUEZ GÓMEZ DE CELIS",
                  "PASTOR JULIÁN",
                  "ELIZO SERRANO",
                  "GIL LÁZARO",
                  "PISARELLO PRADOS",
                  "HERNANZ COSTA",
                  "SÁNCHEZ SERNA",
                  "NAVARRO LACOBA",
                 "ZAMARRÓN MORENO" ]

dataset_df_sin_mesa = dataset_df[~dataset_df["apellidos"].isin(componentes_mesa)]
dataset_df_sin_mesa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12084 entries, 1 to 46857
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   numero     12084 non-null  int64         
 1   fecha      12084 non-null  datetime64[ns]
 2   apellidos  12084 non-null  object        
 3   grupo      12084 non-null  object        
 4   texto      12084 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 566.4+ KB


## Análisis del corpus

En esta sección se muestran algunas estadísticas básicas del corpus.

In [26]:
color_domain = ['ERC', 'PNV', 'PP', 'PSOE', 'UP', 'Vox']
color_range = ['#fbc259', '#426938', '#1e4b8f', '#e30713', '#693065', '#5ac036']
corpus = dataset_df_sin_mesa

#### Número de palabras

In [27]:
nwords = pd.DataFrame({"grupo": corpus["grupo"],
                       "fecha":corpus["fecha"],
                       "n_words":corpus["texto"].str.count(pat = r"(?u)\b\w\w+\b")})

nwords["n_words"].sum()

6799086

#### Palabras por intervención

In [28]:
print(f"Palabras/intervención: {nwords['n_words'].mean()} (std: {nwords['n_words'].std()})")

Palabras/intervención: 562.6519364448858 (std: 705.1843761669896)


In [29]:
alt.Chart(nwords).mark_bar().encode(
    x=alt.X("n_words:Q"),
    y='count():Q',
).properties(width=600, height=150)

alt.Chart(...)

#### Intervenciones por partido

In [30]:


ninterv = corpus.groupby("grupo").count().reset_index()[["grupo", "texto"]].rename(columns={"texto":"count"}).sort_values("count")
ninterv["percent"]=(100*ninterv["count"]/ninterv["count"].sum()).round(2)
ninterv

,grupo,count,percent
1,PNV,960,7.94
0,ERC,1024,8.47
4,UP,1807,14.95
5,Vox,2090,17.30
2,PP,2648,21.91
3,PSOE,3555,29.42


In [31]:
base = alt.Chart(ninterv, title="Intervenciones por partido").encode(
    theta=alt.Theta("count:Q", stack=True, sort="descending"),
    radius=alt.Radius("count:Q", scale=alt.Scale(type="sqrt", zero=True, rangeMin=10)),
    #color=alt.Color("grupo:N"),
    order=alt.Order("count:Q", sort="descending"),
    color=alt.Color("grupo:N", scale=alt.Scale(domain=color_domain,range=color_range)).legend(None),
)

c_ip1 = base.mark_arc(innerRadius=40, stroke="#fff").properties(width=400, height=400)

c_ip2 = base.mark_text(radiusOffset=40, lineBreak=r'\n', color="black").transform_calculate(
    label= alt.datum.grupo + "\\n" + alt.datum.count + "(" + alt.datum.percent + "%)"
    ).encode(text="label:N")

c_ip3 = alt.Chart(ninterv).mark_text(fontSize=16,lineBreak=r'\n').encode(
    text="total:Q"
).transform_joinaggregate(
    total="sum(count):Q"
).transform_calculate(
    label= "TOTAL\\n" + alt.datum.total
)

c_ip = c_ip1 + c_ip2 + c_ip3
c_ip

alt.LayerChart(...)

#### Intervenciones en el tiempo

In [32]:
ninterv_fecha = corpus.groupby("fecha").count().reset_index().rename(columns={"texto":"count"})
ninterv_fecha

,fecha,numero,apellidos,grupo,count
0,2019-12-03,7,7,7,7
1,2020-01-04,24,24,24,24
2,2020-01-05,20,20,20,20
3,2020-01-07,18,18,18,18
4,2020-02-04,37,37,37,37
...,...,...,...,...,...
234,2023-02-15,72,72,72,72
235,2023-02-16,38,38,38,38
236,2023-02-21,31,31,31,31
237,2023-02-22,71,71,71,71


In [33]:
base = alt.Chart(ninterv_fecha, title="Intervenciones al mes").encode(
    x=alt.X('month(fecha):T', title="mes"),
    y=alt.Y('sum(count):Q', title="nº intervenciones"),
    color=alt.Color('year(fecha):N', title="año")
).properties(
    width=600,
    height=100
)




f1 = base.mark_area(interpolate="linear", opacity=0.6, line={"size":0.5}).properties(width=500, height=200)


f1

alt.Chart(...)

## Guardar

In [ ]:
# corpus.to_csv("dataset/corpus.csv")